# [DEMO] Part 1: Data Setup, Feature Engineering & Segment Definition

## Overview
This notebook covers **Phase 1: Data Setup & Initialization** of the LEGO modeling workflow. We will:
1. **Load Portfolio Data**: Import internal portfolio/business data
2. **Load Macroeconomic Variables (MEVs)**: Import external macro data at multiple frequencies
3. **Feature Engineering**: Create derived variables capturing economic drivers
4. **Initialize Segment**: Define the modeling scope and connect data to models

By the end, we'll have a fully integrated `DataManager` and `Segment` object ready for exploratory analysis and model search.

In [ ]:
# Load required libraries for data handling and LEGO framework
import pandas as pd
import Technic as tc
from Technic import TSFM, DumVar

## Step 1: Load Packages and Internal Portfolio Data

**What we're doing**: Import the LEGO framework and read the internal portfolio data (e.g., housing market metrics).

**Key concept**: The internal dataset contains the **target variable** we want to predict, plus any portfolio-specific features. Here, we're modeling `home_price_index` growth.

In [ ]:
# Read internal portfolio data (housing market metrics)
df_internal = pd.read_csv(r'Demo Data/housing_market.csv')
df_internal.head()

,date,home_price_index,new_permits,housing_starts,housing_completions,months_supply,new_home_sales,mortgage_rate,construction_ppi,rent_cpi
0,2000-01-01,100.000,1727,1636,1574,4.3,873,8.06,144.1,180.9
1,2000-02-01,100.571,1692,1737,1677,4.3,856,8.25,144.7,181.3
2,2000-03-01,101.466,1651,1604,1704,4.3,900,8.31,145.4,181.9
3,2000-04-01,102.540,1597,1626,1610,4.4,841,8.23,145.6,182.3
4,2000-05-01,103.701,1543,1575,1682,4.4,857,8.13,144.9,182.8


In [ ]:
# Create target variables: forward-looking growth rates
# 'home_price_GR1' = 1-period ahead growth (used for modeling)
# 'home_price_GR3' = 3-period ahead growth (alternative target for sensitivity)
df_internal['home_price_GR1'] = df_internal['home_price_index'].pct_change().shift(-1)
df_internal['home_price_GR3'] = df_internal['home_price_index'].pct_change(3).shift(-1)

In [ ]:
# Configure sample periods and instantiate TimeSeriesLoader
# These dates define how data is split for training, validation, and forecasting:
# - in_sample_start to in_sample_end: Historical data used for model estimation
# - full_sample_end: Latest historical data point available
# - scen_p0: Scenario projection start date (when we begin out-of-sample forecasting)
ID_config = {
    'in_sample_start':"2006-01-31",
    'in_sample_end':"2023-09-30",
    'full_sample_end':"2025-09-30",
}
int_ldr = tc.TimeSeriesLoader(
    in_sample_start="2006-01-31",
    in_sample_end="2023-09-30",
    full_sample_end="2025-09-30",
    scen_p0="2023-09-30",  # Forecast scenarios begin here
)
int_ldr.load(df_internal, date_col='date')

In [15]:
int_ldr.internal_data

,home_price_index,new_permits,housing_starts,housing_completions,months_supply,new_home_sales,mortgage_rate,construction_ppi,rent_cpi,home_price_GR1,home_price_GR3
date,,,,,,,,,,,
2000-01-31,100.000,1727,1636,1574,4.3,873,8.06,144.100,180.900,0.005710,NaN
2000-02-29,100.571,1692,1737,1677,4.3,856,8.25,144.700,181.300,0.008899,NaN
2000-03-31,101.466,1651,1604,1704,4.3,900,8.31,145.400,181.900,0.010585,0.025400
2000-04-30,102.540,1597,1626,1610,4.4,841,8.23,145.600,182.300,0.011322,0.031122
2000-05-31,103.701,1543,1575,1682,4.4,857,8.13,144.900,182.800,0.011138,0.033410
...,...,...,...,...,...,...,...,...,...,...,...
2025-09-30,328.978,1415,1306,1371,7.9,738,6.56,341.038,438.322,-0.001629,-0.007686
2025-10-31,328.442,1412,1246,1386,7.9,737,6.30,340.292,438.322,0.000000,-0.004374
2025-11-30,328.442,1412,1246,1386,7.9,737,6.17,338.588,438.875,0.000000,-0.001629


## Step 2: Load Macroeconomic Variables (MEVs)

**What we're doing**: Load external macro data at multiple frequencies (quarterly and monthly), and load scenario projections (Base, Adverse, Severely Adverse).

**Key concept**: MEVs are potential predictors of our target variable. The `MEVLoader` consolidates data from different frequencies and attaches scenario projections for forecasting. It will automatically align these with the internal data frequency.

In [ ]:
# Read historical macro data at different frequencies
df_mev_qtr = pd.read_csv(r'Demo Data/macro_quarterly.csv')
df_mev_mth = pd.read_csv(r'Demo Data/macro_monthly.csv')
df_mev_mth.ffill(inplace=True)  # Forward fill to handle missing values

# Read scenario macro data (projected values under different economic conditions)
# These will be used for forecasting under base, adverse, and severely adverse scenarios
df_scen_mev_qtr_base = pd.read_excel(r'Demo Data/macro_scenarios_quarterly.xlsx', sheet_name='baseline').set_index('observation_date').ffill()
df_scen_mev_qtr_adv = pd.read_excel(r'Demo Data/macro_scenarios_quarterly.xlsx', sheet_name='adverse').set_index('observation_date').ffill()
df_scen_mev_qtr_sev = pd.read_excel(r'Demo Data/macro_scenarios_quarterly.xlsx', sheet_name='severely_adverse').set_index('observation_date').ffill()

df_scen_mev_mth_base = pd.read_excel(r'Demo Data/macro_scenarios_monthly.xlsx', sheet_name='baseline').set_index('observation_date').ffill()
df_scen_mev_mth_adv = pd.read_excel(r'Demo Data/macro_scenarios_monthly.xlsx', sheet_name='adverse').set_index('observation_date').ffill()
df_scen_mev_mth_sev = pd.read_excel(r'Demo Data/macro_scenarios_monthly.xlsx', sheet_name='severely_adverse').set_index('observation_date').ffill()

In [7]:
# Create loader for Macro Variables
mev_ldr = tc.MEVLoader()

# load quarterly macro data
mev_ldr.load(
    source=df_mev_qtr,
    date_col='observation_date'
)

# load monthly macro data
mev_ldr.load(
    source=df_mev_mth,
    date_col='observation_date'
)

# Load scenario quarterly macro data
mev_ldr.load_scens(
    {'Base': df_scen_mev_qtr_base,
     'Adv': df_scen_mev_qtr_adv,
     'Sev': df_scen_mev_qtr_sev},
    set_name='Scenario'
)

# Load scenario monthly macro data
mev_ldr.load_scens(
    {'Base': df_scen_mev_mth_base,
     'Adv': df_scen_mev_mth_adv,
     'Sev': df_scen_mev_mth_sev},
    set_name='Scenario'
)

## Step 3: Create DataManager & Feature Engineering

**What we're doing**: Combine internal and macro data into a unified `DataManager`, and create derived features that capture economic relationships.

**Key concept**: Feature engineering is where domain expertise shines. We create variables like yield curve slopes, credit spreads, and ratios that encode economic theory. These become candidates for the automated model search.

**Reference**: See section 3.1 of The LEGO Playbook for feature engineering principles.

In [ ]:
# Create a MEVLoader and register all data sources
mev_ldr = tc.MEVLoader()

# Load historical macro data (MEVLoader handles frequency alignment automatically)
mev_ldr.load(source=df_mev_qtr, date_col='observation_date')  # Quarterly data
mev_ldr.load(source=df_mev_mth, date_col='observation_date')  # Monthly data

# Attach scenario projections (grouped by scenario set name)
mev_ldr.load_scens(
    {'Base': df_scen_mev_qtr_base, 'Adv': df_scen_mev_qtr_adv, 'Sev': df_scen_mev_qtr_sev},
    set_name='Scenario'  # Quarterly scenarios
)
mev_ldr.load_scens(
    {'Base': df_scen_mev_mth_base, 'Adv': df_scen_mev_mth_adv, 'Sev': df_scen_mev_mth_sev},
    set_name='Scenario'  # Monthly scenarios
)

In [ ]:
# Define a feature engineering function to create derived variables
# This function is applied to all data (historical and scenarios) automatically
def new_features(df_mev: pd.DataFrame, df_in: pd.DataFrame):
    """
    Create structural features encoding economic relationships.
    
    Args:
        df_mev: Macro variable dataframe
        df_in: Internal portfolio dataframe
    Returns:
        Modified dataframes with new features added
    """
    
    # =========================================================================
    # A) Rates & Yield Curve Shape (Term Structure Signals)
    # =========================================================================
    # Yield curve slopes capture expectations about future rates and growth
    df_mev['USYC10_2'] = df_mev['USGOV10Y'] - df_mev['USGOV2Y']      # 10-2 slope
    df_mev['USYC10_1'] = df_mev['USGOV10Y'] - df_mev['USGOV1Y']      # 10-1 slope
    df_mev['USYC10_6M'] = df_mev['USGOV10Y'] - df_mev['USGOV6M']    # 10-6M slope
    df_mev['USYC30_10'] = df_mev['USGOV30Y'] - df_mev['USGOV10Y']   # 30-10 slope
    df_mev['USYC5_2'] = df_mev['USGOV5Y'] - df_mev['USGOV2Y']       # 5-2 slope
    # Real rate expectation from term premium
    df_mev['USREAL_TERM_PREM_PROXY'] = df_mev['USGOV10Y'] - df_mev['USGOVR10Y']

    # =========================================================================
    # B) Credit Conditions (Spreads / Wedges)
    # =========================================================================
    # Credit spreads reflect investor risk appetite and borrowing conditions
    df_mev['USCORP_SPRD_BAA_AAA'] = df_mev['USCORPBBB10Y'] - df_mev['USCORPAA10Y']
    df_mev['USCORP_SPRD_BAA_T10'] = df_mev['USCORPBBB10Y'] - df_mev['USGOV10Y']
    df_mev['USCORP_SPRD_AAA_T10'] = df_mev['USCORPAA10Y'] - df_mev['USGOV10Y']
    # Short-term funding spreads (monetary policy transmission)
    df_mev['USCP_FF_SPRD'] = df_mev['USCPF'] - df_mev['USFF']
    df_mev['USPRIME_FF_SPRD'] = df_mev['USPRIME'] - df_mev['USFF']
    df_mev['USIORB_FF_SPRD'] = df_mev['USIORB'] - df_mev['USFF']
    df_mev['USSOFR_FF_SPRD'] = df_mev['USSOFR'] - df_mev['USFF']

    # =========================================================================
    # C) Mortgage Pricing & Household Financing Wedges
    # =========================================================================
    # Mortgage spreads directly affect housing affordability
    df_mev['USMORT30_T10_SPRD'] = df_mev['USMORT30Y'] - df_mev['USGOV10Y']
    df_mev['USMORT15_T10_SPRD'] = df_mev['USMORT15Y'] - df_mev['USGOV10Y']
    df_mev['USARM_T2_SPRD'] = df_mev['USAM51'] - df_mev['USGOV2Y']
    df_mev['USMORT30_15_SPRD'] = df_mev['USMORT30Y'] - df_mev['USMORT15Y']

    # =========================================================================
    # D) Liquidity / Leverage Ratios
    # =========================================================================
    # These ratios capture monetary conditions and household indebtedness
    df_mev['USLIQ_M2_GDP'] = df_mev['USM2'] / df_mev['USNGDP']
    df_mev['USLIQ_M1_M2'] = df_mev['USM1'] / df_mev['USM2']
    df_mev['USCREDIT_CC_INC'] = df_mev['USCC'] / df_mev['USDI']
    df_mev['USCREDIT_CC_GDP'] = df_mev['USCC'] / df_mev['USNGDP']
    df_mev['USCONS_PCE_INC'] = df_mev['USNC'] / df_mev['USDI']
    df_mev['USSAV_DLR'] = df_mev['USDI'] - df_mev['USNC']

    # =========================================================================
    # E) Implied Deflators (Nominal vs. Real Wedges)
    # =========================================================================
    # These capture inflation expectations embedded in nominal/real data
    df_mev['USPCE_IMPLICIT_DEF'] = df_mev['USNC'] / df_mev['USRC']
    df_mev['USDPI_IMPLICIT_DEF'] = df_mev['USDI'] / df_mev['USRPDI']

    # =========================================================================
    # F) Risk Appetite & Relative Valuation Proxies
    # =========================================================================
    # Risk-on/risk-off indicators affect asset demand and valuations
    df_mev['USRISKON_SPX_VIX'] = df_mev['USSP500'] / df_mev['USVIXA']

    # =========================================================================
    # G) Real Estate Cross-Market Relative Pricing
    # =========================================================================
    # Relative valuations between housing and commercial real estate
    df_mev['USCRE_HOUS_REL'] = df_mev['USNCREIF'] / df_mev['USCSH']
    df_mev['USCP_GDP_SHARE'] = df_mev['USCP'] / df_mev['USNGDP']

    return df_mev, df_in

# Create unified DataManager combining internal and macro data
dm = tc.DataManager(
    int_ldr,
    mev_ldr
)

# Apply feature engineering to all data (historical + scenarios)
dm.apply_to_all(new_features)

# Update the variable metadata to categorize new features for documentation
var_update_dict = {
    # A) Rates & curve shape
    'USYC10_2': {'type': 'rate', 'category': 'yield slope'},
    'USYC10_1': {'type': 'rate', 'category': 'yield slope'},
    'USYC10_6M': {'type': 'rate', 'category': 'yield slope'},
    'USYC30_10': {'type': 'rate', 'category': 'yield slope'},
    'USYC5_2': {'type': 'rate', 'category': 'yield slope'},
    'USREAL_TERM_PREM_PROXY': {'type': 'rate', 'category': 'real rate spread'},

    # B) Credit conditions
    'USCORP_SPRD_BAA_AAA': {'type': 'rate', 'category': 'credit spread'},
    'USCORP_SPRD_BAA_T10': {'type': 'rate', 'category': 'credit spread'},
    'USCORP_SPRD_AAA_T10': {'type': 'rate', 'category': 'credit spread'},
    'USCP_FF_SPRD': {'type': 'rate', 'category': 'funding spread'},
    'USPRIME_FF_SPRD': {'type': 'rate', 'category': 'bank pricing spread'},
    'USIORB_FF_SPRD': {'type': 'rate', 'category': 'policy spread'},
    'USSOFR_FF_SPRD': {'type': 'rate', 'category': 'funding spread'},

    # C) Mortgage wedges
    'USMORT30_T10_SPRD': {'type': 'rate', 'category': 'mortgage spread'},
    'USMORT15_T10_SPRD': {'type': 'rate', 'category': 'mortgage spread'},
    'USARM_T2_SPRD': {'type': 'rate', 'category': 'mortgage spread'},
    'USMORT30_15_SPRD': {'type': 'rate', 'category': 'mortgage spread'},

    # D) Liquidity / leverage ratios
    'USLIQ_M2_GDP': {'type': 'level', 'category': 'liquidity ratio'},
    'USLIQ_M1_M2': {'type': 'level', 'category': 'liquidity ratio'},
    'USCREDIT_CC_INC': {'type': 'level', 'category': 'leverage ratio'},
    'USCREDIT_CC_GDP': {'type': 'level', 'category': 'leverage ratio'},
    'USCONS_PCE_INC': {'type': 'level', 'category': 'consumption ratio'},
    'USSAV_DLR': {'type': 'level', 'category': 'savings level'},

    # E) Implied deflators
    'USPCE_IMPLICIT_DEF': {'type': 'level', 'category': 'price level'},
    'USDPI_IMPLICIT_DEF': {'type': 'level', 'category': 'price level'},

    # F) Risk / relative valuation
    'USRISKON_SPX_VIX': {'type': 'level', 'category': 'risk appetite'},

    # G) Real estate cross-market
    'USCRE_HOUS_REL': {'type': 'level', 'category': 'relative valuation'},
    'USCP_GDP_SHARE': {'type': 'level', 'category': 'income share'},
}

dm.update_var_map(var_update_dict)

e:\OneDrive\Git\Project_LEGO\Technic\data.py:1549: UserWarning: apply_to_all(): No scenario internal data for Scenario/Adv; using main internal data as context. Internal scenario updates will be skipped.
  warnings.warn(
e:\OneDrive\Git\Project_LEGO\Technic\data.py:1575: UserWarning: apply_to_all(): Scenario internal data for Scenario/Adv is unavailable; returned internal updates were ignored.
  warnings.warn(
e:\OneDrive\Git\Project_LEGO\Technic\data.py:1549: UserWarning: apply_to_all(): No scenario internal data for Scenario/Base; using main internal data as context. Internal scenario updates will be skipped.
  warnings.warn(
e:\OneDrive\Git\Project_LEGO\Technic\data.py:1575: UserWarning: apply_to_all(): Scenario internal data for Scenario/Base is unavailable; returned internal updates were ignored.
  warnings.warn(
e:\OneDrive\Git\Project_LEGO\Technic\data.py:1549: UserWarning: apply_to_all(): No scenario internal data for Scenario/Sev; using main internal data as context. Internal 

In [ ]:
# ============================================================================
# Step 4: Initialize Segment for Home Price Modeling
# ============================================================================
# The Segment object orchestrates data, configuration, and model search
# for a specific modeling task (target variable + model architecture).

seg_config = {
    'segment_id': 'home_price_GR1',           # Unique identifier for this segment
    'target': 'home_price_GR1',               # Target variable to predict
    'model_type': tc.Growth,                  # Model architecture (Growth for forward-looking growth rates)
    'target_base': 'home_price_index',        # Base series for growth calculations
    'data_manager': dm,                       # The unified data repository created above
    'model_cls': tc.OLS                       # Base model class (standard linear regression)
}

seg = tc.Segment(**seg_config)

print(f"✓ Segment '{seg.segment_id}' initialized successfully")
print(f"  Target variable: {seg.target}")
print(f"  Model type: {seg.model_type.__name__}")
print(f"  In-sample period: {seg.dm.in_sample_start.date()} to {seg.dm.in_sample_end.date()}")